In [6]:
import boto3
from dotenv import load_dotenv
import os

load_dotenv('../.env')
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

# Parámetros del bucket
bucket_name = 'assessment-86fc5eb8'
key = 'raw-data/data.csv'
local_path = '../data/data.csv'

# Crear carpeta si no existe
os.makedirs(os.path.dirname(local_path), exist_ok=True)


# Sesión con credenciales explícitas
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-1' 
)

s3 = session.client('s3')

# Descargar archivo
s3.download_file(bucket_name, key, local_path)